##  Aspect-based Sentiment Analysis (SA)

In [1]:
# Install libraries
!pip install transformers datasets scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

In [2]:
# Imports
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

In [3]:
# Load a pre-trained BERT model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [5]:
# Load IMDb Dataset
dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
# Function to get BERT CLS token embedding
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = bert_model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :]
    return embedding.squeeze().cpu().numpy()

In [9]:
# Shuffle
dataset["train"] = dataset["train"].shuffle(seed=42)
dataset["test"] = dataset["test"].shuffle(seed=42)

# Now Safe to Pick
train_sentences = dataset["train"]["text"][:1000]
train_labels = dataset["train"]["label"][:1000]

test_sentences = dataset["test"]["text"][:300]
test_labels = dataset["test"]["label"][:300]

In [10]:
# Get embeddings
X_train = np.vstack([get_bert_embedding(text) for text in train_sentences])
y_train = np.array(train_labels)

X_test = np.vstack([get_bert_embedding(text) for text in test_sentences])
y_test = np.array(test_labels)

# Train Logistic Regression
clf = LogisticRegression(max_iter=300)
clf.fit(X_train, y_train)


LogisticRegression(max_iter=300)

In [11]:
# Evaluate
y_pred = clf.predict(X_test)
print("\n Classification Report:")
print(classification_report(y_test, y_pred, target_names=["negative", "positive"]))


 Classification Report:
              precision    recall  f1-score   support

    negative       0.77      0.79      0.78       150
    positive       0.78      0.77      0.77       150

    accuracy                           0.78       300
   macro avg       0.78      0.78      0.78       300
weighted avg       0.78      0.78      0.78       300



In [15]:
# Function to predict sentiment for a given sentence
def predict_sentiment(sentence):
    # Get BERT embedding
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = bert_model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    # Predict using the Logistic Regression model
    prediction = clf.predict([embedding])[0]

    # Map prediction back to label
    label = "positive" if prediction == 1 else "negative"
    return label

In [20]:
# Example Sentences
test_examples = [
    ("The battery lasts for two days without charging."),
    ("The camera produces blurry photos."),
    ("The screen is bright and vibrant."),
    ("The sound quality is terrible."),
    ("The latest update improved performance a lot."),
    ("The movie was an absolute masterpiece!"),
    ("I hated every second of that film.")
]


In [21]:
# Predict and display
for sentence in test_examples:
    sentiment = predict_sentiment(sentence)
    print(f"Sentence: \"{sentence}\" ➔ Predicted Sentiment: {sentiment}")

Sentence: "The battery lasts for two days without charging." ➔ Predicted Sentiment: negative
Sentence: "The camera produces blurry photos." ➔ Predicted Sentiment: negative
Sentence: "The screen is bright and vibrant." ➔ Predicted Sentiment: positive
Sentence: "The sound quality is terrible." ➔ Predicted Sentiment: negative
Sentence: "The latest update improved performance a lot." ➔ Predicted Sentiment: positive
Sentence: "The movie was an absolute masterpiece!" ➔ Predicted Sentiment: positive
Sentence: "I hated every second of that film." ➔ Predicted Sentiment: negative
